In [1]:
from urllib.request import urlretrieve
import jieba
import warnings
warnings.filterwarnings("ignore")
"""
url = "https://github.com/fxsjy/jieba/raw/master/extra_dict/dict.txt.big"
urlretrieve(url, "dict.big")
url = "https://github.com/Elwing-Chou/mtkdl/raw/master/ptt_dic.txt"
urlretrieve(url, "dict.ptt")
"""
jieba.set_dictionary("dict.big")
jieba.load_userdict("dict.ptt")

Building prefix dict from D:\onedrive\Documents\data\dict.big ...
Loading model from cache R:\TEMP\jieba.u3fc4963e5b0b71d0127ab7586a4086cb.cache
Loading model cost 1.380 seconds.
Prefix dict has been built successfully.


In [2]:
import pandas as pd
data = pd.read_csv("data.csv", header=0)
data.head()[:1]

,產品名,分數,產品圖片,品牌,分類,評論1,評論2,評論3,評論4,評論5,評論6,評論7,評論8,評論9,評論10
0,三色修容餅 Triple Contour Palette,5.4,https://dg9ugnb21lig7.cloudfront.net/uploads/p...,heme,修容餅,Heme三色修容盤 已經購入一陣子啦～ 用到鐵皮君出沒😆 色號：01 NATIONA...,獲得超多美妝YouTuber 推薦的 heme三色修容餅 以前化妝沒那麼注重修容 都隨便刷刷...,這款修容上市前 我一直都是使用 too cool for school 的修容餅 不得不說h...,"以前沒有使用修容的習慣,後來開始上課學習彩妝以後 經過老師的說明,還有一些美妝YT的影片後 ...",這塊修容餅粉質綿密是屬於沒那麼顯色的，但就是因為不太顯色的特性，很適合修容或製造眼窩及鼻影...,官網表示: 首次推出「三色修容餅」，精選冷暖兩大色調，特調三色色塊圓型均分，可任意沾取混色，...,在IG上抽獎抽到的 顏色還不錯 可惜用沒兩次就不小心摔碎了QQ 沒辦法提供太多心得 只能挖I...,首先要先說他的價格，購入價格是299真的cp很高，對於之前沒用過修容的我也很容易上手，我是用...,買了00號冷色調 最淺色用作眼窩打底 中間色打鼻影兩側 中間跟最深色混刷打在下顎線 覺得很自...,NONE


In [3]:
df = data.copy()
df["combine"] = df[df.columns[5:]].apply(" ".join, axis=1)

In [4]:
import re
def process(content):
    content = re.sub(r'https?:\/\/.*[\r\n]*', '', content)
    punct = set(u''':!),.:;?]}¢'"、。〉》」』】〕〗〞︰︱︳﹐､﹒﹔﹕﹖﹗﹚﹜﹞！），．：；？｜｝︴︶︸︺︼︾﹀﹂﹄﹏､～￠々‖•·ˇˉ―--′’”([{£¥'"‵〈《「『【〔〖（［｛￡￥〝︵︷︹︻︽︿﹁﹃﹙﹛﹝（｛“‘-—_…~/ －＊➜■─★☆=@<>◉é''')
    cut = filter(lambda x: x not in punct, jieba.cut(content))
    return " ".join(cut)
df["combine"] = df["combine"].apply(process)
df["combine"] = df["combine"].str.replace("NONE", "")
df["combine"] = df["combine"].str.replace("的", "")
df["combine"] = df["combine"].str.replace("我", "")
df["combine"] = df["combine"].str.replace("是", "")
df["combine"] = df["combine"].str.replace("買", "")
df["combine"] = df["combine"].str.replace("了", "")
df["combine"] = df["combine"].str.replace("有", "")
df["combine"] = df["combine"].str.replace("很", "")

In [5]:
with open("lines.txt", "w", encoding="utf-8") as f:
    for content in df["combine"]:
        f.write(content)
        f.write("\n")

In [6]:
from gensim.models.word2vec import LineSentence
from gensim.models.word2vec import Word2Vec
sentences = LineSentence("lines.txt")
model = Word2Vec(sentences)

In [7]:
vocab = "武士刀"
model.wv[vocab]
model.wv.most_similar(vocab)

[('哈', 0.9983420968055725),
 ('香港', 0.9979693293571472),
 ('自從', 0.9979409575462341),
 ('曾經', 0.9977555274963379),
 ('三合一', 0.9975197315216064),
 ('火紅', 0.9974365234375),
 ('身邊', 0.9974044561386108),
 ('保濕款', 0.9973937273025513),
 ('打算', 0.9973816275596619),
 ('此生', 0.9973341226577759)]

In [8]:
import numpy as np
import jieba.analyse
encodings = []
for content in df["combine"]:
    tags = jieba.analyse.extract_tags(
        content, 
        topK=None, 
        withWeight=True
    )
    total = np.zeros((100, ))
    weight = 0.0
    for word, tfidf in tags:
        if word in model.wv.vocab:
            total = total + tfidf * model.wv[word]
            weight = weight + tfidf
    encodings.append(total / weight)
print(len(encodings))

867


In [9]:
with open("w2v_vocab.wv", "w", encoding="utf-8") as f:
    f.write("{} {}\n".format(867, 100))
    for i, wv in enumerate(encodings):
        wv = " ".join(map(str, wv))
        f.write("{} {}\n".format(i, wv))

In [10]:
from gensim.models import KeyedVectors
d2v = KeyedVectors.load_word2vec_format("w2v_vocab.wv")

In [11]:
tok = input("Enter(0-866):")
data = df.iloc[int(tok)]
print("品牌：", data["品牌"])
print("分數：", data["分數"])
print("產品名：", data["產品名"])
print("內容：", data["combine"])
print("-" * 30)
tok2, sim = d2v.most_similar(tok)[0]
print("相似度:", sim)
print("-" * 30)
data = df.iloc[int(tok2)]
print("品牌：", data["品牌"])
print("分數：", data["分數"])
print("產品名：", data["產品名"])
print("內容：", data["combine"])

Enter(0-866):20
品牌： 1028
分數： 5.5
產品名： 久耀閃閃打亮盤
內容： 忘   被 版 上 哪一位 網友 給 生火  她 刷 這顆  光澤 讓  好燒 啊 但  對 打亮 最 不能 接受  就 黃 光 而 最 多 網友 開箱  炫光金 就 帶 一點 黃 光 黃皮 人 真 超怕 黃 上加 黃 後來 才 發現 1028 根本 就  出 偏 白光 版  銀 白金 不 知道 到底  他 缺貨 太久 還  一直 眼殘 沒 看到 總之 白 金色 極合 意 就 趁 週六 77 折帶  之前 沒  到 限量 盤 殘念 好 在 1028 又 出  這款 久耀 閃閃 打亮 盤 ❤ 這款 只 單一 色 炫光金 而且 這次  壓紋 超美 ❤ ❤ ❤ ❤  鑽石 光芒  壓紋 光看 心情 就  好 XD 一盒  8g 可以 用到 天荒地老 特價 時 不到 三百元 CP 值  高 ❤ 顏色  炫光金 粉質 算細 不太會 飛粉 之前 1028 出  打亮 餅 都 沒  到 當 在 寶雅 逛  時候  看到  這顆 打亮  以 為  重新 回歸 後來 才 發現 原來  新 產品 這顆 花紋 做 得 漂亮 手背 刷色 摸起來 點 諾諾  刷色 看起來 也  超閃  來看 妝效  沒  到 之前  打亮 餅 所以  也 不 清楚 差 在 哪 其實 少 會 特別注意 開架  打亮 產品 因為 catie  推薦 過 網路上 討論 度 也 蠻高  看到 它  特價 想 說 來 試 看看 而且 容量 跟 粉餅 一樣 足足 8G 掀開 後 鑽石 般 切割 面 閃閃發亮 顏色 為 香檳金 質地 柔滑 金屬 光  光澤感 閃耀 光芒 用 指腹 輕輕 一抹 亮度高 提亮  效果 好 強烈  金屬 光感 讓 五官 更加 立體 終於 被  等到  炫光金 💫 🌟 ✨ 沒想到  也 會 成為 打亮 控  一員 🤩 之前 就 看到 多 人 推薦 1028  打亮 那 時候  2 個 顏色 炫光金 & 暮光橘 不過  去 找  時候 都 只 剩下 暮光橘 顏色  比較 健康 膚色  所以 一直  糾結 到底 要 不要 下手 還好  忍住  😁 😁 這次  名稱 變成 # 1028 久耀 閃閃 打亮 盤 最近 媽咪 再練 手感 想到 就 會 化妝 主要  近視 真 太過 深  眼鏡 拿掉 幾乎 看不

In [12]:
target_data = pd.read_csv("target.csv", header=0)
target_df = target_data.copy()
target_df["content"] = target_df["content"].apply(process)
target_data

,title,content
0,nature自然妝感,你上一次化全妝是什麼時候呢？很多人的答案會是一多個月以前。2021春夏時尚週妝容多以「自然」...
1,wild狂野風格,Hermès 一改以往的裸妝路線，今年首度以自家的口紅亮相時裝週，化妝師Dick Page使...
2,smoke深黑眼線,DIOR秀上模特立體深邃的雙眸，迪奧彩妝創意形象總監Peter Philips表示：「我想以...
3,nature鮮明頰紅,將柔嫩的粉色腮紅大面積暈染兩頰，和模特兒的深棕膚色和奶油色裸唇形成了鮮明的對比，既清新又俏麗...


In [13]:
import numpy as np
import jieba.analyse
encodings = []
for content in target_df["content"]:
    tags = jieba.analyse.extract_tags(
        content, 
        topK=None, 
        withWeight=True
    )
    total = np.zeros((100, ))
    weight = 0.0
    for word, tfidf in tags:
        if word in model.wv.vocab:
            total = total + tfidf * model.wv[word]
            weight = weight + tfidf
    encodings.append(total / weight)
print(len(encodings))

4


In [14]:
with open("target_w2v_vocab.wv", "w", encoding="utf-8") as f:
    f.write("{} {}\n".format(4, 100))
    for i, wv in enumerate(encodings):
        wv = " ".join(map(str, wv))
        f.write("{} {}\n".format(i, wv))

In [15]:
from gensim.models import KeyedVectors
target_d2v = KeyedVectors.load_word2vec_format("target_w2v_vocab.wv")

In [16]:
tok = input("Enter(0-3):")
data = target_df.iloc[int(tok)]
print("風格：", data["title"])
print("內容：", data["content"])
print("-" * 30)
tok2, sim = d2v.most_similar(tok)[0]
print("相似度:", sim)
print("-" * 30)
data = df.iloc[int(tok2)]
print("品牌：", data["品牌"])
print("分數：", data["分數"])
print("產品名：", data["產品名"])
print("內容：", data["combine"])

Enter(0-3):0
風格： nature自然妝感
內容： 你 上 一次 化全妝 是 什麼 時候 呢 很多 人 的 答案 會 是 一 多個 月 以前 2021 春夏 時尚 週妝容 多 以 自然 風格 的 簡單 妝容 亮相 近乎 完美 的 肌膚 和 淡淡的 眼影 使用 輕薄 的 粉底 維持 模特兒 的 自然 膚色 並 利用 液體 唇膏 製造 自然 的 雀斑 妝容 選擇 與 唇色 相近 的 唇膏 使 整體 妝容 維持 一致性 打造 自然 素顏感 一樣 使用 輕薄 的 粉底 保持 了 模特兒 的 自然 膚色 只 使用 微量 的 遮 瑕 來 修飾 肌膚 的 暗 沈及 黑眼圈 並在 顴骨 處 進行 打亮 使 肌膚 看起來 更 有 光澤 膨潤感
------------------------------
相似度: 0.9994322061538696
------------------------------
品牌： M.A.C
分數： 4.9
產品名： 3D調光盤 Hyper Real Glow
內容： 雖然 身邊  打亮  但 總 嫌 顔 色 過於 自然 M A C 出  3D 調 光盤 時  吸引  好像 他們 家  打亮 產品 還蠻 顯色 和 出名  它共出 2 種色盤     # Flash + Awe 還蠻 大  一盤 顔 色 從左到右 裸金 Flash + Awe 玫瑰 金 Rosy Glow 香檳金 Light of the Part 這盤 之前 被燒  七暈 八素  😂 😂 跑  兩間 都 賣完  他們 說  限量  讓 心 都 涼  😭 😭 結果 在 Carousell 剛好 看到 人 賣 他  因為   太多高 光盤 想 清出 一些 全 新品 啊 😍 😍 嘿嘿 來 開箱 早上 超 興奮  😂 😂 這盤  三個 顏色 Flash + Aw Rosy Gold Light 購入   # Flash + Awe 粉白色 調 為  讓 臉上  高級  光澤感 所以 購入 專櫃  打亮 光澤感 細緻 度  珠光  開架 無法 相比  👍 🏻 常 使用 在 臉上  顏色  左邊 跟 中間  顏色 薄薄 使用 就  若隱若現  高級 光澤 搭配 MAC  貓掌 刷 就  完美 ❤ ️ 雖然 打亮 產品 多 而且 用不完 但 就  容易 被 那 道光 吸引 因為 